In [1]:
import pandas as pd
import unicodedata
import requests

# #send to rest API
import json 
import requests
from requests.auth import HTTPBasicAuth

In [2]:
batter_team_2025 = pd.read_csv('../general/stats/batter_team_2025.csv')
def_rank=pd.read_csv('../scrapers/def_scraper/batter_hits_def_rank.csv')
under_hit_rate=pd.read_csv('undernewhits.csv')
over_hit_rate=pd.read_csv('overnewhits.csv')


In [3]:
batter_team_2025.rename(columns={'Team': 'team', 'Player':'player'}, inplace=True)


In [4]:
team_mapping_5 = {
    'CHC': 'CHC',
    'LAD': 'LAD',
    'ARI': 'ARI',
    'PIT': 'PIT',
    'MIA': 'MIA',
    'TOR': 'TOR',
    'BOS': 'BOS',
    'ATH': 'ATH',
    'BAL': 'BAL',
    'SEA': 'SEA',
    'SDP': 'SDP',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'TEX': 'TEX',
    'MIL': 'MIL',
    'WSN': 'WSH',
    'NYY': 'NYY',
    'STL': 'STL',
    'TBR': 'TBR',
    'DET': 'DET',
    'KCR': 'KCR',
    'CLE': 'CLE',
    'COL': 'COL',
    'NYM': 'NYM',
    'CIN': 'CIN',
    'MIN': 'MIN',
    'SFG': 'SFG',
    'HOU': 'HOU',
    'CHW': 'CHW',
    'LAAs': 'LAA'
}

batter_team_2025['team'] = batter_team_2025['team'].map(team_mapping_5)

In [5]:
def_rank.rename(columns={'Team': 'team'}, inplace=True)


In [6]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'ATH',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank['team'] = def_rank['team'].map(team_mapping)

In [7]:
def_rank

,Unnamed: 0,team,Rank
0,0,NYY,1
1,1,HOU,2
2,2,SDP,3
3,3,CIN,4
4,4,TEX,5
5,5,DET,6
6,6,NYM,7
7,7,ATL,8
8,8,MIN,9
9,9,SFG,10


In [8]:
def_rank.rename(columns={'Rank': 'def_rank'}, inplace=True)


In [9]:
#fixing the accentes on the names
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [10]:
batter_team_2025['player'] = batter_team_2025['player'].apply(normalize_name)

In [11]:
#cleaning the # and *
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

batter_team_2025['player'] = batter_team_2025['player'].apply(clean_name_symbols)


In [12]:
# multi_team_flags = ['2TM', '3TM', '4TM', '5TM']
# filtered_df_2 = batter_team_2025[~batter_team_2025['team'].isin(multi_team_flags)].copy()


# # Step 2: Keep only the LAST team listed for each player (i.e., current team)
# batter_team_2025 = filtered_df_2.drop_duplicates(subset='player', keep='last').reset_index(drop=True)

In [13]:
# Set API Key
headers = {
    'X-Api-Key': 'fac7e1a69f2baa47da395bde36dc869c'  # Ensure this is correct
}

# API URL
url = 'https://api.sportsgameodds.com/v2/events/'

next_cursor = None
event_data = []

# Fetch only Over/Under odds for Goals + Assists across all players
while True:
    try:
        response = requests.get(
            url,
            headers=headers,
            params={
                "sportID":"BASEBALL",
                "leagueID": "MLB",
                "oddIDs": "batting_hits-PLAYER_ID-game-ou-over",
                "includeOpposingOddIDs": "true", 
                "oddsAvailable":"true",
                "cursor": next_cursor,
                "include":"teams",
                'started':"false",

            },
        )

        response.raise_for_status()  # Raise an error if the request fails

        data = response.json()
        event_data.extend(data.get("data", []))
        next_cursor = data.get("nextCursor")

        if not next_cursor:
            break

    except requests.exceptions.RequestException as e:
        print("Error fetching events:", e)
        break



In [14]:
event_data

[{'eventID': '31Qcqt0rYjSZKKkvvSYK',
  'sportID': 'BASEBALL',
  'leagueID': 'MLB',
  'type': 'match',
  'teams': {'home': {'teamID': 'PITTSBURGH_PIRATES_MLB',
    'names': {'long': 'Pittsburgh Pirates',
     'short': 'PIT',
     'medium': 'Pirates'},
    'colors': {'primary': '#000000', 'primaryContrast': '#FFFFFF'},
    'statEntityID': 'home'},
   'away': {'teamID': 'CINCINNATI_REDS_MLB',
    'names': {'long': 'Cincinnati Reds', 'short': 'CIN', 'medium': 'Reds'},
    'colors': {'primary': '#C80021', 'primaryContrast': '#FFFFFF'},
    'statEntityID': 'away'}},
  'odds': {'batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over': {'oddID': 'batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over',
    'opposingOddID': 'batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under',
    'marketName': 'Spencer Horwitz Hits Over/Under',
    'statID': 'batting_hits',
    'statEntityID': 'SPENCER_HORWITZ_1_MLB',
    'periodID': 'game',
    'betTypeID': 'ou',
    'sideID': 'over',
    'playerID': 'SPENCER_HORWITZ_1_MLB',

In [15]:
# Initialize an empty list for structured data
odds_list = []

for event in event_data:
    event_id = event.get("eventID", "Unknown Event")
    teams = sorted(set(player_data.get("teamID") for player_data in event.get("players", {}).values() if player_data.get("teamID")))
    
    odds = event.get("odds", {})
    
    for odd_id, odd_object in odds.items():
        base_row = {
            "eventID": event_id,
            "teams": ", ".join(teams),
            "oddID": odd_object.get("oddID"),
            "opposingOddID": odd_object.get("opposingOddID"),
            "marketName": odd_object.get("marketName"),
            "playerID": odd_object.get("statEntityID"),
            "side": odd_object.get("sideID"),
            "line": odd_object.get("bookOverUnder", odd_object.get("fairOverUnder", "N/A")),
            "score": odd_object.get("score", "N/A"),
        }
        
        # Extract bookmaker-specific odds into separate rows
        by_bookmaker = odd_object.get("byBookmaker", {})
        
        for bookmaker, book_data in by_bookmaker.items():
            row = base_row.copy()
            row["bookmaker"] = bookmaker
            row["odds"] = book_data.get("odds", "N/A")
            row["overUnder"] = book_data.get("overUnder", "N/A")
            odds_list.append(row)

# Convert to Pandas DataFrame
df_odds = pd.DataFrame(odds_list)

# Print the DataFrame
df_odds.head(5)

,eventID,teams,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder
0,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Hits Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,draftkings,-160,0.5
1,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Hits Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,espnbet,-160,0.5
2,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Hits Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,fanatics,-145,0.5
3,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Hits Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,fliff,-160,0.5
4,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over,Spencer Horwitz Hits Over/Under,SPENCER_HORWITZ_1_MLB,under,0.5,N/A,draftkings,+120,0.5


In [16]:
# df_odds['overUnder'].value_counts()

In [17]:
# row = df_odds.loc[
#     (df_odds['marketName'] == 'Carlos Correa Hits Over/Under') & 
#     (df_odds['bookmaker'].str.lower() == 'fanduel')
# ]

# if not row.empty:
#     line_value = row['overUnder'].values[0]
#     odds_value = row['odds'].values[0]  # Replace 'odds' with actual column name if different
#     print(f"Line: {line_value}, Odds: {odds_value}")
# else:
#     print("Carlos Correa market at FanDuel not found.")


In [18]:
#getting the last 3 letters of the teams collumn to get MLB
df_odds['sport'] = df_odds['teams'].str[-3:]

In [19]:
# Split into two new columns
df_odds[['team_1', 'team_2']] = df_odds['teams'].str.split(', ', expand=True)
# Remove the "_MLB" suffix from both columns using .loc
df_odds.loc[:, 'team_1'] = df_odds['team_1'].str.replace('_MLB', '', regex=False)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].str.replace('_MLB', '', regex=False)
# Drop the original 'teams' column
df_odds = df_odds.drop(columns=['teams'])

In [20]:
#extract the players name from marketName
df_odds['player'] = df_odds['marketName'].str.extract(r'^(.*?)\s*Hits')

In [21]:
# Define the mapping dictionary
team_mapping = {
    'ARIZONA_DIAMONDBACKS': 'ARI',
    'ATLANTA_BRAVES': 'ATL',
    'BALTIMORE_ORIOLES': 'BAL',
    'BOSTON_RED_SOX': 'BOS',
    'CHICAGO_CUBS': 'CHC',
    'CHICAGO_WHITE_SOX': 'CHW',
    'CINCINNATI_REDS': 'CIN',
    'CLEVELAND_GUARDIANS': 'CLE',
    'COLORADO_ROCKIES': 'COL',
    'DETROIT_TIGERS': 'DET',
    'HOUSTON_ASTROS': 'HOU',
    'KANSAS_CITY_ROYALS': 'KCR',
    'LOS_ANGELES_ANGELS': 'LAA',
    'LOS_ANGELES_DODGERS': 'LAD',
    'MIAMI_MARLINS': 'MIA',
    'MILWAUKEE_BREWERS': 'MIL',
    'MINNESOTA_TWINS': 'MIN',
    'NEW_YORK_METS': 'NYM',
    'NEW_YORK_YANKEES': 'NYY',
    'OAKLAND_ATHLETICS': 'ATH',
    'PHILADELPHIA_PHILLIES': 'PHI',
    'PITTSBURGH_PIRATES': 'PIT',
    'SAN_DIEGO_PADRES': 'SDP',
    'SAN_FRANCISCO_GIANTS': 'SFG',
    'SEATTLE_MARINERS': 'SEA',
    'STLOUIS_CARDINALS': 'STL',
    'TAMPA_BAY_RAYS': 'TBR',
    'TEXAS_RANGERS': 'TEX',
    'TORONTO_BLUE_JAYS': 'TOR',
    'WASHINGTON_NATIONALS': 'WSH'}


# Apply the mapping
df_odds.loc[:, 'team_1'] = df_odds['team_1'].map(team_mapping)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].map(team_mapping)



df_odds.head(4)

,eventID,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder,sport,team_1,team_2,player
0,31Qcqt0rYjSZKKkvvSYK,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Hits Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,draftkings,-160,0.5,MLB,CIN,PIT,Spencer Horwitz
1,31Qcqt0rYjSZKKkvvSYK,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Hits Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,espnbet,-160,0.5,MLB,CIN,PIT,Spencer Horwitz
2,31Qcqt0rYjSZKKkvvSYK,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Hits Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,fanatics,-145,0.5,MLB,CIN,PIT,Spencer Horwitz
3,31Qcqt0rYjSZKKkvvSYK,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-over,batting_hits-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Hits Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,fliff,-160,0.5,MLB,CIN,PIT,Spencer Horwitz


In [22]:
#making the line collumn an integer
df_odds['overUnder'] = pd.to_numeric(df_odds['overUnder'], errors='coerce')

In [23]:
df_odds['bet']='hits'

In [24]:
df_odds=df_odds[['player','side','overUnder','bookmaker','odds','sport','team_1','team_2','bet']]

In [25]:
# Merge on the 'player' column
df_odds = df_odds.merge(
    batter_team_2025[['player', 'team']],
    on='player',
    how='left'
)


In [26]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team
0,Spencer Horwitz,over,0.5,draftkings,-160,MLB,CIN,PIT,hits,PIT
1,Spencer Horwitz,over,0.5,espnbet,-160,MLB,CIN,PIT,hits,PIT
2,Spencer Horwitz,over,0.5,fanatics,-145,MLB,CIN,PIT,hits,PIT
3,Spencer Horwitz,over,0.5,fliff,-160,MLB,CIN,PIT,hits,PIT
4,Spencer Horwitz,under,0.5,draftkings,+120,MLB,CIN,PIT,hits,PIT
...,...,...,...,...,...,...,...,...,...,...
747,Sam Huff,over,0.5,fanduel,-160,MLB,KCR,SFG,hits,SFG
748,Christian Koss,over,0.5,fanduel,-155,MLB,KCR,SFG,hits,SFG
750,Dairon Blanco,over,0.5,fanduel,-155,MLB,KCR,SFG,hits,KCR
751,Hunter Renfroe,over,0.5,fanduel,-125,MLB,KCR,SFG,hits,KCR


In [27]:
# Step 1: Identify the "odd one out" team to find the defensive team
df_odds['def_team'] = df_odds.apply(lambda row: row['team_2'] if row['team'] == row['team_1'] else row['team_1'], axis=1)

In [28]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team,def_team
0,Spencer Horwitz,over,0.5,draftkings,-160,MLB,CIN,PIT,hits,PIT,CIN
1,Spencer Horwitz,over,0.5,espnbet,-160,MLB,CIN,PIT,hits,PIT,CIN
2,Spencer Horwitz,over,0.5,fanatics,-145,MLB,CIN,PIT,hits,PIT,CIN
3,Spencer Horwitz,over,0.5,fliff,-160,MLB,CIN,PIT,hits,PIT,CIN
4,Spencer Horwitz,under,0.5,draftkings,+120,MLB,CIN,PIT,hits,PIT,CIN
...,...,...,...,...,...,...,...,...,...,...,...
747,Sam Huff,over,0.5,fanduel,-160,MLB,KCR,SFG,hits,SFG,KCR
748,Christian Koss,over,0.5,fanduel,-155,MLB,KCR,SFG,hits,SFG,KCR
750,Dairon Blanco,over,0.5,fanduel,-155,MLB,KCR,SFG,hits,KCR,SFG
751,Hunter Renfroe,over,0.5,fanduel,-125,MLB,KCR,SFG,hits,KCR,SFG


In [29]:
#drop redundant collumns that are useless
df_odds = df_odds.drop(columns=['team_1', 'team_2'], errors='ignore')

In [30]:
# Rename the team column before merging (if needed)
def_rank = def_rank.rename(columns={"team": "def_team"})

In [31]:
# Merge on the 'player' column
df_odds = df_odds.merge(def_rank, on='def_team', how='left')
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,bet,team,def_team,Unnamed: 0,def_rank
0,Spencer Horwitz,over,0.5,draftkings,-160,MLB,hits,PIT,CIN,3,4
1,Spencer Horwitz,over,0.5,espnbet,-160,MLB,hits,PIT,CIN,3,4
2,Spencer Horwitz,over,0.5,fanatics,-145,MLB,hits,PIT,CIN,3,4
3,Spencer Horwitz,over,0.5,fliff,-160,MLB,hits,PIT,CIN,3,4
4,Spencer Horwitz,under,0.5,draftkings,+120,MLB,hits,PIT,CIN,3,4
...,...,...,...,...,...,...,...,...,...,...,...
747,Sam Huff,over,0.5,fanduel,-160,MLB,hits,SFG,KCR,10,11
748,Christian Koss,over,0.5,fanduel,-155,MLB,hits,SFG,KCR,10,11
750,Dairon Blanco,over,0.5,fanduel,-155,MLB,hits,KCR,SFG,9,10
751,Hunter Renfroe,over,0.5,fanduel,-125,MLB,hits,KCR,SFG,9,10


In [32]:
#making game collumn
df_odds['game'] = df_odds['team'] + ' vs ' + df_odds['def_team']


In [33]:
df_odds2=df_odds.dropna()

In [34]:
df_odds3 = df_odds2.drop(columns=['team', 'def_team'])
df_odds3

,player,side,overUnder,bookmaker,odds,sport,bet,Unnamed: 0,def_rank,game
0,Spencer Horwitz,over,0.5,draftkings,-160,MLB,hits,3,4,PIT vs CIN
1,Spencer Horwitz,over,0.5,espnbet,-160,MLB,hits,3,4,PIT vs CIN
2,Spencer Horwitz,over,0.5,fanatics,-145,MLB,hits,3,4,PIT vs CIN
3,Spencer Horwitz,over,0.5,fliff,-160,MLB,hits,3,4,PIT vs CIN
4,Spencer Horwitz,under,0.5,draftkings,+120,MLB,hits,3,4,PIT vs CIN
...,...,...,...,...,...,...,...,...,...,...
747,Sam Huff,over,0.5,fanduel,-160,MLB,hits,10,11,SFG vs KCR
748,Christian Koss,over,0.5,fanduel,-155,MLB,hits,10,11,SFG vs KCR
750,Dairon Blanco,over,0.5,fanduel,-155,MLB,hits,9,10,KCR vs SFG
751,Hunter Renfroe,over,0.5,fanduel,-125,MLB,hits,9,10,KCR vs SFG


In [35]:
#splitting overs and unders
overs = df_odds3[(df_odds3['side'].str.lower() == 'over') & (df_odds3['def_rank'] > 15)]
unders = df_odds3[(df_odds3['side'].str.lower() == 'under') & (df_odds3['def_rank'] < 16)]
# overs

In [36]:
overs = overs.merge(over_hit_rate, on='player', how='left')
unders = unders.merge(under_hit_rate, on='player', how='left')
# overs

In [37]:
overs_2=overs.dropna()
unders_2=unders.dropna()

In [38]:
#making odds collumn numeric
unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')

C:\Users\Joshua\AppData\Local\Temp\ipykernel_112892\2110421890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
C:\Users\Joshua\AppData\Local\Temp\ipykernel_112892\2110421890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')


In [39]:
#making decimal odds
unders_2['decimal_odds'] = unders_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)
overs_2['decimal_odds'] = overs_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)
# overs_2

C:\Users\Joshua\AppData\Local\Temp\ipykernel_112892\3651279232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['decimal_odds'] = unders_2['odds'].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_112892\3651279232.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['decimal_odds'] = overs_2['odds'].apply(


In [40]:
#making profit
overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100

C:\Users\Joshua\AppData\Local\Temp\ipykernel_112892\3093691362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
C:\Users\Joshua\AppData\Local\Temp\ipykernel_112892\3093691362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100


In [41]:
overs_2

,player,side,overUnder,bookmaker,odds,sport,bet,Unnamed: 0_x,def_rank,game,Unnamed: 0_y,1_hits_rate_over,2_hits_rate_over,3_hits_rate_over,4_hits_rate_over,decimal_odds,profit
3,Nico Hoerner,over,0.5,draftkings,-200,MLB,hits,25,25,CHC vs MIA,117.0,0.750000,0.283333,0.066667,0.000000,1.500000,50.000000
4,Nico Hoerner,over,0.5,espnbet,-215,MLB,hits,25,25,CHC vs MIA,117.0,0.750000,0.283333,0.066667,0.000000,1.465116,46.511628
5,Nico Hoerner,over,0.5,fanatics,-190,MLB,hits,25,25,CHC vs MIA,117.0,0.750000,0.283333,0.066667,0.000000,1.526316,52.631579
6,Nico Hoerner,over,0.5,fliff,-215,MLB,hits,25,25,CHC vs MIA,117.0,0.750000,0.283333,0.066667,0.000000,1.465116,46.511628
11,Dansby Swanson,over,0.5,draftkings,-165,MLB,hits,25,25,CHC vs MIA,32.0,0.655172,0.241379,0.034483,0.000000,1.606061,60.606061
12,Dansby Swanson,over,0.5,espnbet,-165,MLB,hits,25,25,CHC vs MIA,32.0,0.655172,0.241379,0.034483,0.000000,1.606061,60.606061
13,Dansby Swanson,over,0.5,fanatics,-145,MLB,hits,25,25,CHC vs MIA,32.0,0.655172,0.241379,0.034483,0.000000,1.689655,68.965517
14,Dansby Swanson,over,0.5,fliff,-175,MLB,hits,25,25,CHC vs MIA,32.0,0.655172,0.241379,0.034483,0.000000,1.571429,57.142857
22,Seiya Suzuki,over,0.5,draftkings,-190,MLB,hits,25,25,CHC vs MIA,136.0,0.666667,0.352941,0.058824,0.019608,1.526316,52.631579
23,Seiya Suzuki,over,0.5,espnbet,-190,MLB,hits,25,25,CHC vs MIA,136.0,0.666667,0.352941,0.058824,0.019608,1.526316,52.631579


In [42]:
#edge for overs

def calculate_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_hits_rate_over']
    elif row['overUnder'] == 1.5:
        rate = row['2_hits_rate_over']
    else:
        return None  # or np.nan, if you want to leave it blank for other lines

    # Expected Value formula
    return (rate * row['profit']) - ((1 - rate) * 100)

# Apply the formula to each row
overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)


C:\Users\Joshua\AppData\Local\Temp\ipykernel_112892\3262990477.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)


In [43]:
#edge for unders
def calculate_under_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_hits_rate_under']
    elif row['overUnder'] == 1.5:
        rate = row['2_hits_rate_under']
    else:
        return None  # or np.nan for unsupported lines

    return ((1 - rate) * row['profit']) - (rate * 100)

# Apply to each row
unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)



C:\Users\Joshua\AppData\Local\Temp\ipykernel_112892\3648321810.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)


In [44]:
overs_2 = overs_2[overs_2['edge'] > 20]

In [45]:
unders_2 = unders_2[unders_2['edge'] > 20]

In [46]:
#merging overs_2 on top of unders_2
final_df = pd.concat([overs_2, unders_2], ignore_index=True)


In [47]:
final_df=final_df[['player','side','overUnder','bookmaker','odds','sport','bet','edge','game']]

In [48]:
final_df['bet'] = final_df['side'].astype(str) + ' ' + final_df['overUnder'].astype(str) + ' ' + final_df['bet'].astype(str)


In [49]:
#need to add + in front of positive odds 

In [50]:
#rename bookmaker
final_df = final_df.rename(columns={'bookmaker': 'book'})

In [51]:
#divide the edge in half
final_df['edge'] = final_df['edge'] / 2

In [52]:
#round the edge to 1 decimal
final_df['edge'] = final_df['edge'].round(1)

In [53]:
final_df=final_df[['book','sport','player','game','bet','odds','edge']]

In [54]:
final_df = final_df.sort_values(by='edge', ascending=False).reset_index(drop=True)


In [55]:
#putting + in front of the ones that dont have it
final_df['odds'] = final_df['odds'].apply(lambda x: f"+{x}" if x > 0 else str(x))

final_df

,book,sport,player,game,bet,odds,edge
0,caesars,MLB,Christopher Morel,TBR vs HOU,under 0.5 hits,+146,20.3
1,draftkings,MLB,Christopher Morel,TBR vs HOU,under 0.5 hits,+145,20.0
2,espnbet,MLB,Christopher Morel,TBR vs HOU,under 0.5 hits,+145,20.0
3,fanatics,MLB,Christopher Morel,TBR vs HOU,under 0.5 hits,+140,18.6
4,fliff,MLB,Christopher Morel,TBR vs HOU,under 0.5 hits,+140,18.6
5,draftkings,MLB,Adley Rutschman,BAL vs MIL,under 0.5 hits,+150,17.1
6,draftkings,MLB,Jackson Holliday,BAL vs MIL,under 0.5 hits,+175,17.0
7,draftkings,MLB,Gavin Lux,CIN vs PIT,under 0.5 hits,+170,16.4
8,fliff,MLB,Gavin Lux,CIN vs PIT,under 0.5 hits,+170,16.4
9,fliff,MLB,Jackson Holliday,BAL vs MIL,under 0.5 hits,+170,15.8


In [56]:
import json
import requests

# Convert your DataFrame to JSON
data_json = json.loads(final_df.to_json(orient="records"))

# Endpoint URL
url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_hits"

# Headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request without authentication
response = requests.post(url, headers=headers, json=data_json)

# Debugging output
print("Status Code:", response.status_code)
print("Response Text:", response.text)


Status Code: 200
Response Text: {"message":"Data received","data":[{"book":"caesars","sport":"MLB","player":"Christopher Morel","game":"TBR vs HOU","bet":"under 0.5 hits","odds":"+146","edge":20.3},{"book":"draftkings","sport":"MLB","player":"Christopher Morel","game":"TBR vs HOU","bet":"under 0.5 hits","odds":"+145","edge":20},{"book":"espnbet","sport":"MLB","player":"Christopher Morel","game":"TBR vs HOU","bet":"under 0.5 hits","odds":"+145","edge":20},{"book":"fanatics","sport":"MLB","player":"Christopher Morel","game":"TBR vs HOU","bet":"under 0.5 hits","odds":"+140","edge":18.6},{"book":"fliff","sport":"MLB","player":"Christopher Morel","game":"TBR vs HOU","bet":"under 0.5 hits","odds":"+140","edge":18.6},{"book":"draftkings","sport":"MLB","player":"Adley Rutschman","game":"BAL vs MIL","bet":"under 0.5 hits","odds":"+150","edge":17.1},{"book":"draftkings","sport":"MLB","player":"Jackson Holliday","game":"BAL vs MIL","bet":"under 0.5 hits","odds":"+175","edge":17},{"book":"draftkin

In [57]:
# # Convert DataFrame to JSON (ensure it's a Python list, not a string)
# data_json = json.loads(final_df.to_json(orient="records"))  # ✅ Converts string to Python object

# # WordPress API URL
# url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_hits"

# # Authentication
# auth = HTTPBasicAuth("josheskragmailcom", "UhiM WLwm dEOK T171 RUkT Sw2F")
# # auth = HTTPBasicAuth("josheskragmailcom", "BFgx z2iv nW9I voXn 1X51 fjRq")
# # Headers
# headers = {
#     "Content-Type": "application/json"
# }

# # Send POST request (requests will handle JSON encoding automatically)
# response = requests.post(url, headers=headers, auth=auth, json=data_json)  # ✅ FIX: Use json= instead of data=

# # Debugging Output
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)

In [58]:
#YOU CAN IGNORE EVERYTHING BELOW THIS, THIS IS FOR ME TO KEEP TRACK OF THE BETS LOCALLY

In [59]:
# Step 1: Sort to ensure reproducibility in tie cases
df_sorted = final_df.sort_values('edge')

# Step 2: Define a function to get the row with median edge for each group
def get_median_row(group):
    median_edge = group['edge'].median()
    # Get the row closest to the median (or first in case of exact match/ties)
    return group.iloc[(group['edge'] - median_edge).abs().argsort().iloc[0]]

# Step 3: Apply it to each group
df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)

# Step 4: Reset index if needed
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

C:\Users\Joshua\AppData\Local\Temp\ipykernel_112892\1266780210.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)


,book,sport,player,game,bet,odds,edge
0,espnbet,MLB,Adley Rutschman,BAL vs MIL,under 0.5 hits,+145,15.8
1,espnbet,MLB,Christopher Morel,TBR vs HOU,under 0.5 hits,+145,20.0
2,fanatics,MLB,Gavin Lux,CIN vs PIT,under 0.5 hits,+160,13.9
3,fliff,MLB,Jackson Holliday,BAL vs MIL,under 0.5 hits,+170,15.8
4,draftkings,MLB,Jake Meyers,HOU vs TBR,under 0.5 hits,+140,11.1
5,fliff,MLB,Ryan O'Hearn,BAL vs MIL,under 0.5 hits,+125,11.4
6,fanatics,MLB,Santiago Espinal,CIN vs PIT,under 0.5 hits,+160,11.0
7,caesars,MLB,Taylor Walls,TBR vs HOU,under 0.5 hits,-106,12.6


In [60]:
# Make sure odds column is numeric
df_filtered['odds'] = pd.to_numeric(df_filtered['odds'], errors='coerce')
# Convert American odds to decimal odds
def american_to_decimal(odds):
    if pd.isna(odds):
        return None
    if odds > 0:
        return round((odds / 100) + 1, 3)
    else:
        return round((100 / abs(odds)) + 1, 3)

# Apply conversion
df_filtered['decimal_odds'] = df_filtered['odds'].apply(american_to_decimal)

In [61]:
df_filtered['profit'] = (df_filtered['decimal_odds'] * 100) - 100
df_filtered

,book,sport,player,game,bet,odds,edge,decimal_odds,profit
0,espnbet,MLB,Adley Rutschman,BAL vs MIL,under 0.5 hits,145,15.8,2.450,145.0
1,espnbet,MLB,Christopher Morel,TBR vs HOU,under 0.5 hits,145,20.0,2.450,145.0
2,fanatics,MLB,Gavin Lux,CIN vs PIT,under 0.5 hits,160,13.9,2.600,160.0
3,fliff,MLB,Jackson Holliday,BAL vs MIL,under 0.5 hits,170,15.8,2.700,170.0
4,draftkings,MLB,Jake Meyers,HOU vs TBR,under 0.5 hits,140,11.1,2.400,140.0
5,fliff,MLB,Ryan O'Hearn,BAL vs MIL,under 0.5 hits,125,11.4,2.250,125.0
6,fanatics,MLB,Santiago Espinal,CIN vs PIT,under 0.5 hits,160,11.0,2.600,160.0
7,caesars,MLB,Taylor Walls,TBR vs HOU,under 0.5 hits,-106,12.6,1.943,94.3


In [62]:
df_filtered.to_csv('median_hits_bets.csv')